# [모듈 2-1] SageMaker Pipelines 사용하기
이 노트북에서는 아래와 같은 작업을 수행합니다.
- 데이터 준비
- Pipeline 정의
- 데이터 전처리: Processing Step 이용


In [2]:
import sagemaker

sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [7]:
pipeline_prefix = 'autoglueon'
s3_data_path = f's3://{bucket}/{pipeline_prefix}/raw'
s3_config_path = f's3://{bucket}/{pipeline_prefix}/config'

sm_dataset_path = '../data/raw'
sm_claims_data_path = f'{sm_dataset_path}/claims.csv'
sm_customers_data_path = f'{sm_dataset_path}/customers.csv'
sm_config_path = f'../config'

In [8]:
print("claim_data_path: ", sm_claims_data_path)
print("customer_data_path: ", sm_customers_data_path)
print("s3_data_path: ", s3_data_path)

claim_data_path:  ../data/raw/claims.csv
customer_data_path:  ../data/raw/customers.csv
s3_data_path:  s3://sagemaker-ap-northeast-2-457597599702/autoglueon/raw


In [9]:
s3_claims_data_path = sagemaker.s3.S3Uploader.upload(
    local_path = sm_claims_data_path, 
    desired_s3_uri = s3_data_path
)
print("claims data path in S3: ", s3_claims_data_path)

s3_customers_data_path = sagemaker.s3.S3Uploader.upload(
    local_path = sm_customers_data_path, 
    desired_s3_uri = s3_data_path
)
print("customers data path in S3: ", s3_customers_data_path)


s3_config_path = sagemaker.s3.S3Uploader.upload(
    local_path = sm_config_path, 
    desired_s3_uri = s3_config_path
)
print("config path in S3: ", s3_config_path)


claims data path in S3:  s3://sagemaker-ap-northeast-2-457597599702/autoglueon/raw/claims.csv
customers data path in S3:  s3://sagemaker-ap-northeast-2-457597599702/autoglueon/raw/customers.csv
config path in S3:  s3://sagemaker-ap-northeast-2-457597599702/autoglueon/config


# 2.2 전처리 스텝 개발

In [10]:
import os
import boto3
import pandas as pd
from IPython.display import display as dp

In [11]:
df_customers = pd.read_csv(sm_customers_data_path)
df_customers.head()

,policy_id,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_state,policy_deductable,policy_annual_premium,policy_liability,customer_zip,customer_gender,customer_education,auto_year
0,1,54,94,0,1,WA,750,3000,25/50,99207,Unkown,Associate,2006
1,2,41,165,0,1,CA,750,2950,15/30,95632,Male,Bachelor,2012
2,3,57,155,0,1,CA,750,3000,15/30,93203,Female,Bachelor,2017
3,4,39,80,0,1,AZ,750,3000,30/60,85208,Female,Advanced Degree,2020
4,5,39,60,0,1,CA,750,3000,15/30,91792,Female,High School,2018


In [12]:
df_claims = pd.read_csv(sm_claims_data_path)
df_claims.head()

,policy_id,driver_relationship,incident_type,collision_type,incident_severity,authorities_contacted,num_vehicles_involved,num_injuries,num_witnesses,police_report_available,injury_claim,vehicle_claim,total_claim_amount,incident_month,incident_day,incident_dow,incident_hour,fraud
0,1,Spouse,Collision,Front,Minor,None,2,0,0,No,71600,8913.668763,80513.668763,3,17,6,8,0
1,2,Self,Collision,Rear,Totaled,Police,3,4,0,Yes,6400,19746.724395,26146.724395,12,11,2,11,0
2,3,Self,Collision,Front,Minor,Police,2,0,1,Yes,10400,11652.969918,22052.969918,12,24,1,14,0
3,4,Child,Collision,Side,Minor,None,2,0,0,No,104700,11260.930936,115960.930936,12,23,0,19,0
4,5,Self,Collision,Side,Major,Police,2,1,0,No,3400,27987.704652,31387.704652,5,8,2,8,0


In [13]:
%%writefile src/preprocess.py

import argparse
import os
import requests
import tempfile
import subprocess, sys

import pandas as pd
import numpy as np
from glob import glob

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import logging
import logging.handlers

def _get_logger():
    '''
    로깅을 위해 파이썬 로거를 사용
    # https://stackoverflow.com/questions/17745914/python-logging-module-is-printing-lines-multiple-times
    '''
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  

logger = _get_logger()


def split_train_test(df, test_ratio=0.1):
    '''
    두 개의 데이터 세트로 분리
    '''
    total_rows = df.shape[0]
    train_end = int(total_rows * (1 - test_ratio))
    
    train_df = df[0:train_end]
    test_df = df[train_end:]
    
    return train_df, test_df


def get_dataframe(base_preproc_input_dir, file_name_prefix ):    
    '''
    파일 이름이 들어가 있는 csv 파일을 모두 저장하여 데이터 프레임을 리턴
    '''
    
    input_files = glob('{}/{}*.csv'.format(base_preproc_input_dir, file_name_prefix))
    #claim_input_files = glob('{}/dataset*.csv'.format(base_preproc_input_dir))    
    logger.info(f"input_files: \n {input_files}")    
    
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(base_preproc_input_dir, "train"))
        
    raw_data = [ pd.read_csv(file, index_col=0) for file in input_files ]
    df = pd.concat(raw_data)
   
    logger.info(f"dataframe shape \n {df.shape}")    
    logger.info(f"dataset sample \n {df.head(2)}")        
    #logger.info(f"df columns \n {df.columns}")    
    
    return df


def convert_type(raw, cols, type_target):
    '''
    해당 데이터 타입으로 변경
    '''
    df = raw.copy()
    
    for col in cols:
        df[col] = df[col].astype(type_target)
    
    return df
    

if __name__ =='__main__':
    
    ################################
    #### 커맨드 인자 파싱   
    #################################        
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--base_output_dir', type=str, default="/opt/ml/processing/output")
    parser.add_argument('--base_preproc_input_dir', type=str, default="/opt/ml/processing/input")   
    parser.add_argument('--split_rate', type=float, default=0.1)       
    parser.add_argument('--label_column', type=str, default="fraud")       
    # parse arguments
    args = parser.parse_args()     
    
    logger.info("######### Argument Info ####################################")
    logger.info(f"args.base_output_dir: {args.base_output_dir}")
    logger.info(f"args.base_preproc_input_dir: {args.base_preproc_input_dir}")    
    logger.info(f"args.label_column: {args.label_column}")        
    logger.info(f"args.split_rate: {args.split_rate}")            

    base_output_dir = args.base_output_dir
    base_preproc_input_dir = args.base_preproc_input_dir
    label_column = args.label_column    
    split_rate = args.split_rate

    #################################        
    #### 두개의 파일(claim, customer) 을 로딩하여 policy_id 로 조인함  ########
    #################################    
    
    logger.info(f"\n### Loading Claim Dataset")
    claim_df = get_dataframe(base_preproc_input_dir,file_name_prefix='claim' )        
    
    logger.info(f"\n### Loading Customer Dataset")    
    customer_df = get_dataframe(base_preproc_input_dir,file_name_prefix='customer' )            
    
    df = customer_df.join(claim_df, how='left')
    logger.info(f"### dataframe merged with customer and claim: {df.shape}")


    #################################    
    #### 카테고리 피쳐를 원핫인코딩  
    #################################    
    
    logger.info(f"\n ### Encoding: Category Features")    
    categorical_features = df.select_dtypes(include=['object']).columns.values.tolist()    
    #categorical_features = ['driver_relationship']    
    logger.info(f"categorical_features: {categorical_features}")            

    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]
    )
    
    preprocess = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, categorical_features)
        ],
        sparse_threshold = 0, # dense format 으로 제공
    )

    X_pre_category = preprocess.fit_transform(df)
    

    # 원핫인코딩한 컬럼의 이름 로딩
    # Ref: Sklearn Pipeline: Get feature names after OneHotEncode In ColumnTransformer,  https://stackoverflow.com/questions/54646709/sklearn-pipeline-get-feature-names-after-onehotencode-in-columntransformer
    
    processed_category_features = preprocess.transformers_[0][1].named_steps['onehot'].get_feature_names(categorical_features)
    #logger.info(f"processed_category_features: {processed_category_features}")
#    print(X_pre)
    
    ###############################
    ### 숫자형 변수 전처리 
    ###############################
    
    logger.info(f"\n ### Encoding: Numeric Features")        
    
    float_cols = df.select_dtypes(include=['float64']).columns.values
    int_cols = df.select_dtypes(include=['int64']).columns.values
    numeric_features = np.concatenate((float_cols, int_cols), axis=0).tolist()
    
    logger.info(f"int_cols: \n{int_cols}")    
    logger.info(f"float_cols: \n{float_cols}")        
    #logger.info(f"numeric_features: \n{numeric_features}")

    # 따로 스케일링은 하지 않고, 미싱 값만 중간값을 취함
    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
           # ("scaler", StandardScaler())
        ]
    )

    numeric_preprocessor = ColumnTransformer(
        transformers=[
            ("cat", numeric_transformer, numeric_features)
        ],
        sparse_threshold = 0,
    )

    X_pre_numeric = numeric_preprocessor.fit_transform(df)    

    
    ###############################
    ### 전처리 결과 결합 ####
    ###############################
    
    logger.info(f"\n ### Handle preprocess results")            
    
    # 전처리 결과를 데이터 프레임으로 생성
    category_df = pd.DataFrame(data=X_pre_category, columns=processed_category_features)
    numeric_df = pd.DataFrame(data=X_pre_numeric, columns=numeric_features)    

    full_df = pd.concat([numeric_df, category_df ], axis=1)
    
    # float 타입을 int 로 변경
    full_df = convert_type(full_df, cols=int_cols, type_target='int')
    full_df = convert_type(full_df, cols=processed_category_features, type_target='int')    
    
    # label_column을 맨 앞으로 이동 시킴
    full_df = pd.concat([full_df[label_column], full_df.drop(columns=[label_column])], axis=1)
    
    ###############################    
    # 훈련, 테스트 데이터 세트로 분리 및 저장
    ###############################
    
    train_df, test_df = split_train_test(full_df, test_ratio=split_rate)    
    train_df.to_csv(f"{base_output_dir}/dataset/train.csv", index=False)
    test_df.to_csv(f"{base_output_dir}/dataset/test.csv", index=False)    

    logger.info(f"preprocessed train shape \n {train_df.shape}")        
    logger.info(f"preprocessed test shape \n {test_df.shape}")            

    # logger.info(f"preprocessed train path \n {base_output_dir}/train/train.csv")
    logger.info(f"\n ### Final result for train dataset ")    
    logger.info(f"preprocessed train sample \n {train_df.head(2)}")

Writing src/preprocess.py


In [14]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

framework_version = "0.23-1"
processing_instance_type = "ml.m5.xlarge"
processing_instance_count = 1
split_rate = 0.1

sklearn_processor = SKLearnProcessor(
    framework_version = framework_version,
    instance_type = processing_instance_type,
    instance_count = processing_instance_count,
    base_job_name = "sklearn-fraud-process",
    role = role,
)

In [15]:
sklearn_processor.run(
    code = "src/preprocess.py",
    inputs = [ProcessingInput(source = s3_data_path, destination = "/opt/ml/processing/input")],
    outputs = [ProcessingOutput(output_name = "dataset",
                                source = "/opt/ml/processing/output/dataset")],
    arguments = ['--split_rate', f"{split_rate}"]
)



Job Name:  sklearn-fraud-process-2022-07-25-08-58-29-293
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-457597599702/autoglueon/raw', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-457597599702/sklearn-fraud-process-2022-07-25-08-58-29-293/input/code/preprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'dataset', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-2-457597599702/sklearn-fraud-process-2022-07-25-08-58-29-293/output/dataset', 'LocalPath': '/opt/ml/processing/output/dataset', 'S3UploadMode': 'EndOfJob'}}]
...................

In [16]:
sklearn_processor.latest_job.outputs[0].destination

's3://sagemaker-ap-northeast-2-457597599702/sklearn-fraud-process-2022-07-25-08-58-29-293/output/dataset'

In [17]:
sagemaker.s3.S3Downloader.download(s3_uri = sklearn_processor.latest_job.outputs[0].destination,
                                   local_path = '../data/preprocessed')

In [18]:
preprocessed_train_path = '../data/preprocessed/train.csv'
preprocessed_train_df = pd.read_csv(preprocessed_train_path)
preprocessed_train_df.head()

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0


# 모델 빌딩 파이프라인의 스텝 생성
## 모델 빌딩 파이프라인의 파라미터 정의
파이프라인에서 사용 할 파라미터를 정의합니다. 파이프라인을 실행할 때 파라미터를 이용하여 실행 조건을 커스터마이징 할 수 있습니다.
지원하는 파라미터 타입은 다음과 같습니다.

* `ParameterString` - 파이썬 타입에서 `str` 
* `ParameterInteger` - 파이썬 타입에서 `int` 
* `ParameterFloat` - 파이썬 타입에서 `float` 

디폴트 값을 지정할 수 있으며, 파이프라인 실행 시 재지정 할 수도 있습니다.

In [19]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)
input_data = ParameterString(
    name="InputData",
    default_value=s3_data_path,
)

## 전처리 스텝 프로세서 정의

In [20]:
# !pip install --upgrade sagemaker

In [38]:
import sagemaker
sagemaker.__version__

'2.100.0'

In [22]:
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = "0.23-1"
processing_instance_type = "ml.m5.xlarge"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-fraud-process",
    role=role,
)

# 전처리 스텝 정의
Processing Step에서는 다음과 같은 인자들을 지정합니다.
* name: 스텝 명
* processor: 전처리 프로세서
* inputs: 입력 데이터 S3 경로
* outputs: 처리결과가 저장 될 Docker 안의 경로
* job arguments: 사용자 정의 인자
* code: 전처리 코드 경로

보다 자세한 내용은 링크를 확인하세요
[처리 단계, Processing Step](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)

In [23]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    

step_process = ProcessingStep(
    name="Fraud-Preprocessing",
    processor=sklearn_processor,
    code= "src/preprocess.py",
    inputs = [ProcessingInput(source = s3_data_path, destination = "/opt/ml/processing/input")],
    outputs = [ProcessingOutput(output_name = "dataset",
                                source = "/opt/ml/processing/output/dataset")],
    job_arguments = ['--split_rate', f"{split_rate}"]
)

# 파이프라인 정의 및 실행
생성한 단계들을 하나의 파이프라인으로 조합하고 실행할 수 있습니다.
파이프라인은 name, parapeters, steps 인자가 필수로 필요합니다. 
여기서 파이프라인의 name은 Account와 Region에 대해 유일해야 합니다.

주의사항:
- 정의에 사용한 모든 파라미터가 존재해야 합니다.
- 파이프라인으로 전달된 단계(step)들은 실행순서와는 무관합니다. SageMaker Pipeline은 단계가 실행되고 완료될 수 있도록 의존관계를를 해석합니다.
- [알림] 정의한 stpes 이 복수개이면 복수개를 기술합니다. 만약에 step 간에 의존성이 있으면, 명시적으로 기술하지 않아도 같이 실행 됩니다.

In [24]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = pipeline_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        input_data,
    ],
    steps=[step_process],
)

In [25]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-238312515155/sm-autoglueon-pipeline-base/data'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Fraud-Preprocessing',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
     'ContainerArguments': ['--split_rate', '0.1'],
     'ContainerEntrypoint': ['python3',
      '/opt/ml/processing/input/code/preprocess.py']},
    'RoleArn': 'arn:aws:iam::238312515155:role/ser

## 파이프라인 정의를 등록하고 실행하기

In [26]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [27]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:238312515155:pipeline/sm-autoglueon-pipeline-base',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:238312515155:pipeline/sm-autoglueon-pipeline-base/execution/5ovkwrorpqeb',
 'PipelineExecutionDisplayName': 'execution-1658556036042',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 7, 23, 6, 0, 35, 935000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 7, 23, 6, 0, 35, 935000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:238312515155:user-profile/d-no5jmo24zdtn/shimdx-c39',
  'UserProfileName': 'shimdx-c39',
  'DomainId': 'd-no5jmo24zdtn'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:238312515155:user-profile/d-no5jmo24zdtn/shimdx-c39',
  'UserProfileName': 'shimdx-c39',
  'DomainId': 'd-no5jmo24zdtn'},
 'ResponseMetadata': {'RequestId': '046cf723-e952-44e1-9da8-eae91b6e8638',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-

In [28]:
execution.wait()

In [29]:
execution.list_steps()

[{'StepName': 'Fraud-Preprocessing',
  'StartTime': datetime.datetime(2022, 7, 23, 6, 0, 37, 28000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 7, 23, 6, 5, 5, 414000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:238312515155:processing-job/pipelines-5ovkwrorpqeb-fraud-preprocessing-bofkyfacai'}}}]